<a href="https://colab.research.google.com/github/Zen-Teng/Stylos/blob/main/cleaner_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.5 MB/s eta 0:00:00


In [2]:
import pdfplumber

def read_pdf(file_path):
    text = ""

    with pdfplumber.open(file_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                text += f"\n--- Page {page_number} ---\n"
                text += page_text

    return text


In [3]:
from google.colab import files
uploaded = files.upload()

pdf_path = next(iter(uploaded))  # gets the uploaded filename
pdf_text = read_pdf(pdf_path)

print(pdf_text)

#For multiple upload
# for pdf_path in uploaded.keys():
#     print(f"Reading {pdf_path}")
#     pdf_text = read_pdf(pdf_path)
#     print(pdf_text)

Saving Class Participation Assessment.pdf to Class Participation Assessment.pdf

--- Page 1 ---
Class Participation
Course: DSA4262 Sense-making Case Analysis: Health & Medicine
Due date: 16 April 2026
Weightage: 10% of total score
1. Motivation
This course has a different setup from your usual courses. You will mostly learn by doing, and
through discussion with the instructors, guest lecturers, and your group members. Therefore, it is
really crucial that you “show up” in class, not just physically but also actively engaging with the
content. Remember, this is a simulation of a real-world data scientist role, where it’s safe to
experiment and “fail”. It is therefore also important to create a supportive and respectful
atmosphere in the class.
2. Description
Class participation will be assessed both quantitatively and qualitatively. We will not just count
the number of times students speak up or raise their hand, but also consider the quality of their
contributions. Students that listen